# Creative machine learning - Machine Learning

### Author: Philippe Esling (esling@ircam.fr)

In this course we will cover
1. A [first definition](#definition) on the concept of machine learning
2. An introduction to a simple problem of [linear regression](#regression)
4. A detailed implementation of [simple linear regression](#linear)
3. An explanation on [model capacity and overfitting](#capacity)
4. Two [exercises](#exercises) to explore regression and classification
4. An introduction to the [audio datasets](#audio) that we will use

<a id="definition"></a>
## Defining machine learning

In all natural process, there exists complex relations between sets $\mathcal{X} \mapsto \mathcal{Y}$. This can relate some objects with their names, or a cause to a consequence. In most cases, _we do not know the precise relations_ between these sets, all we have is _observations_ such as pairs $(x,y)$, composed of input data $x \in \mathcal{X}$, which have a corresponding expected output $y \in \mathcal{Y}$. The overarching goal of machine learning is to approximate such _unknown processes_ as a function $\mathcal{F}_{\theta}$, which _transforms_ input data $x$ into output data $y$.


<center>
<img src="images/01_machine_learning_basic.png" align="center"/>
</center>

Hence, machine learning aims to understand and model the relationship between some (usually complex and high-dimensional) inputs $\mathbf{x}\in\mathcal{X}\subset\mathbb{R}^{\mathcal{X}}$ and outputs $\mathbf{y}\in\mathcal{Y}\subset\mathbb{R}^{\mathcal{Y}}$, given by a set of data examples $\mathcal{D}=\left\{(x_1,y_1),\cdots,(x_N,y_N)\right\}$. This is achieved by defining a parametric model $f_{\mathbf{\theta}}\in\mathcal{F}$ inside a family of functions $\mathcal{F}$, which depends on parameters $\mathbf{\theta} \in \mathbf{\Theta}$ and that could approximate the underlying relationship. The _learning_ aspect refers to the adjustment of the parameters $\mathbf{\theta}$ in order to obtain the best approximation of the given task
$$
\begin{equation}
f_{\mathbf{\theta}}(\mathbf{x}) = \bar{\mathbf{y}}\approx \mathbf{y}.
\end{equation}
$$

Hence, the major elements that we have to define in any machine learning problems are
1. **Dataset** : $\mathcal{D}=\left\{(x_1,y_1),\cdots,(x_N,y_N)\right\}$. This dataset has to be representative of the relation $f:\mathcal{X} \mapsto \mathcal{Y}$ that we are looking to model
2. **Model** : Our parametric approximation $\bar{\mathbf{y}} = f_{\mathbf{\theta}}(\mathbf{x})$, where the choice of family $f_{\mathbf{\theta}}\in\mathcal{F}$ is critical
3. **Loss** : $\mathcal{L}\left( \bar{\mathbf{y}}, \mathbf{y} \mid f_{\theta}, \theta \right)$ allows to measure the amount of errors made by our model
4. **Optimization** : Method to find $\theta^{*}\in\Theta$ so that our model minimizes the loss
$$\theta^{*}= \underset{\theta}{\text{argmin }} \mathcal{L}\left( \bar{\mathbf{y}}, \mathbf{y} \mid f_{\theta}, \theta \right)$$


To observe this idea in simple setups, we are going to use the `numpy` library and also initialize the homemade course library `cml` and style for future plotting and exercise. We also set the random generator to a fixed point with `rng = np.random.RandomState(1)`, to ensure reproducibility

In [157]:
# Base imports
import numpy as np
import matplotlib.pyplot as plt
from cml.plot import initialize_bokeh
from cml.panel import initialize_panel
from jupyterthemes.stylefx import set_nb_theme
from bokeh.io import show
initialize_bokeh()
initialize_panel()
set_nb_theme("onedork")
rng = np.random.RandomState(1)

Loading BokehJS ...

<a id="regression"></a>
## Simple learning problem

Imagine that a certain process somewhere follows the form of a quadratic relationship

$$
 y = a x^{2} + bx + c 
$$

In this case, all the **unknown parameters** are that of a polynomial model, therefore we have $\theta = \{a, b, c\}$. However, this is clearly an ideal (clean) case, whereas in natural observations, there might be some noise in our observations
$$
 y = a x^{2} + bx + c +\epsilon \quad \mbox{with} \quad \epsilon \in [-0.1, 0.1]
$$

An example of such noisy observations for different parameters is given below

In [158]:
# Parameters to our function
eps = 0.1
a, b, c = 5, 2, 0
# Generating the corresponding data
x = np.linspace(0, 1, 100)
poly = np.poly1d([a, b, c])
epsilon = np.random.uniform(-eps, eps, x.shape)
y = poly(x) + epsilon

In [159]:
from cml.plot import center_plot, scatter
plot = (center_plot(scatter(x, y, title="Simple quadratic problem", toolbar_location="left")))
plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'3a6d76db-ad66-4a55-be10-7df6525f9d8d': {'version…

Now our main problem is that this function can follow different types of parameters

In [160]:
params = [[5, -5, 4], [-2, 1, 0], [0.1, 1, 1]]
# Generating the x axis
x = np.linspace(0, 1, 100)
plots = []
for p in range(len(params)):
    poly = np.poly1d(params[p])
    epsilon = np.random.uniform(-eps, eps, x.shape)
    y = poly(x) + epsilon
    plots.append(scatter(x, y, title="Problem "+(str(p+1))))

In [161]:
from bokeh.models import Div
from bokeh.layouts import column, row
plot = center_plot(column(Div(text = "Observing different problems", styles={'font-size': '250%'}), column(*plots)))
plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'c2adbdf2-e5db-47c4-88ca-d9f47f9cc6fa': {'version…

In real-life settings, this function can also have different levels of noise, as exemplified in the following code.

In [162]:
params = [3, 0, 1]
noise_levels = [0.1, 1.0, 8.0]
# Generating the x axis
x = np.linspace(0, 1, 100)
plots = []
for p in range(len(noise_levels)):
    poly = np.poly1d(params)
    epsilon = np.random.uniform(-noise_levels[p],noise_levels[p],x.shape)
    y = poly(x) + epsilon
    plots.append(scatter(x, y, title="Problem "+(str(p+1))))

In [163]:
plot = center_plot(column(Div(text = "Different amounts of noise", styles={'font-size': '250%'}), column(*plots)))
plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'11926e01-536b-47b9-ac50-f52de4a61d19': {'version…

To summarize, we will have some observations of a function, and we would like to optimize a function that gets as close as possible to the real function that generated this data. Here, we plot the real function and also _subsample_ our number of observations (having only a few points to find the corresponding function)

In [164]:
# Generating the data and subsampling
x_all = np.linspace(0, 1, 100); x_plot = np.linspace(0, 1, 100)
rng.shuffle(x_all); x = np.sort(x_all[:50])
poly = np.poly1d([3,0,1])
# Adding some external noise
epsilon = np.random.uniform(-0.2,0.2,x.shape)
y = poly(x)+ epsilon
p = scatter(x, y, title="Learning problem with groundtruth")
p.line(x_plot, poly(x_plot), line_width=6, line_alpha=0.6, color="green", legend_label=r"True function")
plot = (center_plot(p))
plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'4b685ba8-1262-448b-8d4a-0a90f13afdf6': {'version…

### Summarizing our observations (interactive)



In [165]:
from cml.tasks import RegressionPolynomial
explorer = RegressionPolynomial()
explorer.render()

BokehModel(combine_events=True, render_bundle={'docs_json': {'9eab9f61-a850-40d3-b04a-b6c0863df875': {'version…

## Using learning libraries (`scikit-learn`)

To get a first grip on what machine learning does, we will rely on the `scikit-learn` library. This contains already coded models and learning procedure, that will allow us to _learn_ the parameters of this unknown function.

Here we already know that we want to use a `PolynomialFeatures` model to perfom `LinearRegression` and that this polynomial should be of degree 2.

In [166]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
# Our data to fit
X = x[:, np.newaxis]
# Degree of our polynomial
degree = 30;
# Create our polynomial model for regression
model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
# Fit the parameters of this model
model.fit(X, y);

Now that we have trained the model, we can perform _predictions_ from it, meaning that we can infer the output of the function at values that we did not observe originally.

In [167]:
# Inference points (not observed)
X_plot = x_plot[:, np.newaxis]
# Predict the values
y_plot = model.predict(X_plot)
# Compute the error of our model at observed points
Y_model_err = np.sqrt(np.mean(np.square(y-model.predict(X))))
print(f'Model error : {Y_model_err}')
# Plot the result
p = scatter(x, y, title="Training a scikit-learn model")
p.line(x_plot, poly(x_plot), line_width=6, line_alpha=0.6, color="red", legend_label=r"Trained model")
plot = (center_plot(p))
plot

Model error : 0.09574434258990291


BokehModel(combine_events=True, render_bundle={'docs_json': {'0c2386f4-ce67-4efd-910f-47d651313631': {'version…

In [168]:
from cml.tasks import RegressionPolynomialSolver
explorer = RegressionPolynomialSolver()

In [169]:
def solve(x, y, degree):
    X = x[:, np.newaxis]
    # Create our polynomial model for regression
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    # Fit the parameters of this model
    model.fit(X, y)
    # Predict the values
    x_predict = np.linspace(np.min(x), np.max(x), 200)[:, np.newaxis]
    y_model = model.predict(x_predict)
    return x_predict[:, 0], y_model #np.array(jnp.zeros(y_model.shape))

explorer.solve = solve
explorer.render()

BokehModel(combine_events=True, render_bundle={'docs_json': {'5102e9f5-9bc8-487d-8066-eabeba93f1bb': {'version…

<a id="linear"></a>
# Simple linear regression 

As discussed previously, regression allows to model the relationships that exist between inputs $\mathbf{x}\in\mathbb{R}^{n}$ and a continuous output $y\in\mathbb{R}$. In the case of **linear** regression, we assume that the data that we observe comes from a linear relationship in the input, so that
$$y = w_0 + w_1x_1 + w_2x_2 + ... + w_nx_n + \epsilon$$
with $\epsilon$ exhibiting the *observation noise* (also called *residual error*) that is always present in measurements.

To understand how we could learn a model approximating this relationship, we start with the case of *simple* linear regression, where $\mathbf{x}\in\mathbb{R}$. This implies that our model will follow
$$\bar{y} = w_0 + w_1 x$$

We will measure the errors made by our model by using the Mean Squared Error (MSE) loss, defined as 
$$\mathcal{L}_{MSE}\left( \bar{\mathbf{y}},\theta \right) = \sum_{i=1}^{n} \left| y_{i} - \bar{y}_{i} \right|^{2} = \sum_{i=1}^{n} \left| y_{i} - (w_{0} + w_{1} x_{i}) \right|^{2}$$

Then our goal is to find the most adequate set of parameters $\theta = \{w_{0}, w_{1}\}$, which are those that minimize the MSE loss defined previously. Therefore, we aim to obtain
$$\theta^{*}= \underset{\theta}{\text{argmin }} \mathcal{L}\left( \bar{\mathbf{y}}, \mathbf{y} \mid f_{\theta}, \theta \right)$$

To do so, we will implement the **gradient descent** algorithm discussed in the course.

## Manual implementation - `NumPy`

We start by performing a *full manual implementation*, in the sense that we need to manually derive the gradient in order to apply the gradient descent updates. To do so, we will rely on [NumPy](https://numpy.org/), which is a fundamental library for numerical computing offering support for N-dimensional arrays and scientific computing tasks, such as linear algebra, statistical analysis, and matrix manipulation. We strongly encourage you to learn NumPy through the set of [tutorials](https://numpy.org/learn/). For the sake of this introductory tutorial, we will provide the explanation for all of the functions that we will use in this first exercise. After that exercise, we will assume for the rest of the course that knowledge of Numpy should be found online. 

We start by import libraries (`NumPy`) and set a random seed to ensure that the random number generator produces always a reproducible series of random numbers.

**Used functions**
- `np.random.seed`: sets the seed for the NumPy random number generator. [Documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.seed.html)

In [170]:
import numpy as np
np.random.seed(42)

In [171]:
?np.random.seed

Docstring:
seed(seed=None)

Reseed the singleton RandomState instance.

Notes
-----
This is a convenience, legacy function that exists to support
older code that uses the singleton RandomState. Best practice
is to use a dedicated ``Generator`` instance rather than
the random variate generation methods exposed directly in
the random module.

See Also
--------
numpy.random.Generator
Type:      builtin_function_or_method

### Generate a synthetic dataset.

For the sake of this exercise, we will generate the data ourselves, so that we know the true values that we are looking for in advance. Hence, we define a linear relationship following
$$y = w^{t}_0 + w^{t}_1 x + \epsilon$$

**Used functions**
- `np.random.rand`: generates an array of random numbers uniformly distributed over [0, 1). [Documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html)
- `np.random.randn`: generates an array of random numbers from the standard normal distribution (mean 0, variance 1). [Documentation](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)
- `np.ones`: generates an array of ones with a specified shape. [Documentation](https://numpy.org/doc/stable/reference/generated/numpy.ones.html)
- `np.c_`: concatenates arrays along the second axis. [Documentation](https://numpy.org/doc/stable/reference/generated/numpy.c_.html)

In [172]:
import numpy as np
true_w0 = 2
true_w1 = 3
n_obs = 100
x = [0, 1]
# Input to model
x = np.linspace(x[0], x[1], n_obs + 1)
# Output (target)
y = true_w1 * x + true_w0

In [173]:
def mse_loss(y_bar, y_true):
    return np.sum((y_true - y_bar) ** 2)

In [174]:
true_w0 = 2
true_w1 = 3
x_min = 0
x_max = 1
n_obs = 100
x = np.linspace(x_min, x_max, n_obs)
y = true_w1 * x + true_w0 + eps

In [175]:
def mse_loss(y, y_bar):
    return np.sum((y - y_bar) ** 2)

### Computing the MSE loss

We will measure the errors made by our model by using the Mean Squared Error (MSE) loss, defined as 
$$\mathcal{L}_{MSE}\left( \bar{\mathbf{y}},\theta \right) = \sum_{i=1}^{n} \left| y_{i} - \bar{y}_{i} \right|^{2} = \sum_{i=1}^{n} \left| y_{i} - (w_{0} + w_{1} x_{i}) \right|^{2}$$

This will allow us to evaluate the performances of our model, but is also the basis for the following gradient descent algorithm.

**Used functions**
- `np.sum`: computes the sum of the provided array (optionally across a provided `axis`). [Documentation](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)

In [176]:
def mse_loss(y, y_bar):
    return np.sum((y - y_bar) ** 2)

### Implement the gradient descent algorithm.

As seen in the course Initialize the weight $w_1$ and bias $w_0$ to small random values
1. Evaluate the predictions made by our model 
$$\hat{y} = w_{1}x + w_0$$
2. Compute the mean squared error (MSE) loss between the predicted values and the ground truth labels: 
$$\mathcal{L}_{MSE} = \frac{1}{n} \sum_{i=1}^n (\bar{y_i} - y_i)^2$$
3. Compute the gradients of the loss with respect to the weight and bias (see derivation in the course)
$$\frac{\partial \mathcal{L}}{\partial w_1} = \sum_{i} 2 * (\bar{y}_{i} - y_{i})*x_{i}$$
$$\frac{\partial \mathcal{L}}{\partial w_0} = \sum_{i} 2 * (\bar{y}_{i} - y_{i})$$
4. Update the weights and bias using the gradients and a learning rate $\eta$: 
$$w_1 \leftarrow w_1 - \eta \frac{\partial \mathcal{L}}{\partial w_1}$$ 
$$w_0 \leftarrow w_0 - \eta \frac{\partial \mathcal{L}}{\partial w_0}$$

In [177]:
n_iter = 1000
loss = np.zeros(n_iter)
eta = 1e-3
def model(x, w0, w1):
    return x * w1 + w0
w0 = np.random.randn()
w1 = np.random.randn()
for i in range(n_iter):
    y_bar = model(x, w0, w1)
    l_mse = mse_loss(y, y_bar)
    #print(f'w0 = {w0}, w1 = {w1}, loss = {l_mse}')
    dldw0 = np.sum(2 * (y_bar - y))
    w0 = w0 - eta * dldw0
    dldw1 = np.sum(2 * (y_bar - y) * x)
    w1 = w1 - eta * dldw1
    loss[i] = l_mse
import matplotlib.pyplot as plt
print(w0)
print(w1)

2.100001299981006
2.999997574364702


In [178]:
# Parameters
n_iter = 1000
lr = 0.001
def gradient_descent(x, y, n_iter, lr):
    # Initialize the parameters
    w_1 = np.random.randn()
    w_0 = np.random.randn()
    # Perform gradient descent
    for i in range(n_iter):
        # 1. Calculate the predictions
        y_bar = w_1 * x + w_0
        # 2. Compute the loss
        loss = mse_loss(y, y_bar)
        # 3. Calculate the gradients
        dw_1 = 2 * np.sum((y_bar - y) * x)
        dw_0 = 2 * np.sum((y_bar - y))
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0
    return w_1, w_0
w_1, w_0 = gradient_descent(x, y, n_iter, lr)
print(w_1)
print(w_0)

2.9999978254581143
2.1000011654114497


### Visualize the results.

We can see how well our model fits to the observed data by plotting it against the observed samples.

In [179]:
p = scatter(x, y, title="Simple linear regression")
p.line(x, w_1 * x + w_0, line_width=6, line_alpha=0.6, color="red", legend_label=r"Learned model")
plot = (center_plot(p))
plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'6a89c93a-6a4b-46a0-97c2-0b5a562c5794': {'version…

### Observing our solution interactively

In the following code, you can observe the behavior of the model by playing interactively with the properties of the original problem. 

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> Note that each change in the properties of the original problem requires to run the gradient descent algorithm entirely each time.

</div>

In [180]:
from cml.tasks import RegressionLinearSolver
explorer = RegressionLinearSolver()
def solve(x, y):
    w_1, w_0 = gradient_descent(x, y, n_iter, lr)
    x_predict = np.linspace(np.min(x), np.max(x), 200)
    y_model = w_1 * x_predict + w_0
    return np.array(x_predict), np.array(y_model) #np.array(np.zeros(y_model.shape))

explorer.solve = solve
explorer.render()

BokehModel(combine_events=True, render_bundle={'docs_json': {'e7c86be3-db79-43a1-b891-0144054593c1': {'version…

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Going further
> **Exercise 1 \[Learning rate\]**: Experiment with different learning rates (e.g., 0.001, 0.01, 0.1, 1) and observe how they affect the convergence of the gradient descent algorithm. Plot the convergence history (cost function value vs. iteration) for each learning rate.

After doing the code it was noticed that learning rates from 0.01 and upwards would diverge. The smaller the learning rate value is the longer it takes to find the optimal parameters. However, larger values risk divergence.

> **Exercise 2 \[Initialization\]**: Experiment with different initializations of the coefficients (zeros, random values, etc.) and observe their impact on the convergence of the gradient descent algorithm.

With an appropriate learning rate value, the closer the initial parameters are to the ground truth, the faster the loss converges.

> **Exercise 3 \[Variants\]**: Implement and compare different variants of gradient descent, such as stochastic gradient descent (SGD) and mini-batch gradient descent. Analyze their convergence properties and computational efficiency.

Both Batch Gradient Descent and Mini-Batch Gradient Descent achieve a very low loss rapidly however the mini-batch gradient descent seems to converge to a larger loss value. The SGD is likely very cost efficient but the loss decreases slowly compared to the other two methods. To analyze the computation efficiency each function will be timed. The loss calculation was removed during timing to just have a timing over the computation of the gradient descent method. Surprisingly Batch Gradient Descent is not only the best results but also the best performance. I imagine this is because of my implementation, that requires shuffling or deciding a random value every iteration and for loops in python are very costly?

> **Exercise 4 \[Regularization\]**: Implement Lasso and Ridge regression with gradient descent by incorporating the regularization terms into the cost function and gradient calculations. Compare their performance with the standard linear regression implementation and analyze their impact on the learned coefficients.


</div>

In [181]:
##############
# EXERCISE 1 #
##############
# All these packages are already in the course's requirements.txt
from bokeh.plotting import figure
from bokeh.palettes import Dark2_5 as palette
import itertools

# Parameters
n_iter = 1000
lr = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

# Modified MSE for data with multiple lr values
def mMSE(y, y_bar):
    return np.sum((y - y_bar) ** 2, axis=0)

# Modified Gradient Descent
def mgd(x, y, n_iter, lr):
    # Initialize the parameters for each lr with the same value
    w_1 = np.ones_like(lr) * np.random.randn()
    w_0 = np.ones_like(lr) * np.random.randn()

    # Modify input data to repeat every column
    # Each column will be used for a different lr
    x = np.array([x] * len(lr)).T
    y = np.array([y] * len(lr)).T
    # Store loss for each iteration and different lr
    loss = np.zeros((n_iter, len(lr))) 

    # Perform gradient descent
    for i in range(n_iter):
        # 1. Calculate the predictions
        y_bar = w_1 * x + w_0
        # 2. Compute the loss
        loss[i, :] = mMSE(y, y_bar)
        # 3. Calculate the gradients
        dw_1 = 2 * np.sum((y_bar - y) * x, axis=0)
        dw_0 = 2 * np.sum(y_bar - y, axis=0)
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0
    return loss # Return only the loss for plots

loss_hist = mgd(x, y, n_iter, lr)
# Remove divergent values such as inf, -inf or NaN (which give the warning)
loss_plot = np.where(~np.isfinite(loss_hist), None, loss_hist) 
# create a new plot with a title and axis labels
# add y_range=[0,<small value>] to see values that converge
p = figure(title="Convergence History", x_axis_label='Iteration', y_axis_label='Error', x_range=[0, n_iter]) 
colors = itertools.cycle(palette) # list of colors
for n in range(len(lr)):
    p.line(np.arange(n_iter), loss_hist[:,n], line_width=4, line_alpha=0.6, legend_label=str(lr[n]), color=next(colors))
plot = (center_plot(p))
plot

C:\Users\garam\AppData\Local\Temp\ipykernel_3044\2809584415.py:15: RuntimeWarning: overflow encountered in square
  return np.sum((y - y_bar) ** 2, axis=0)
c:\Users\garam\OneDrive\Documentos\Github Repo\cml\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\garam\AppData\Local\Temp\ipykernel_3044\2809584415.py:33: RuntimeWarning: invalid value encountered in multiply
  y_bar = w_1 * x + w_0


BokehModel(combine_events=True, render_bundle={'docs_json': {'56294069-7147-40b3-8c6d-0852d02f55c5': {'version…

In [182]:
##############
# EXERCISE 2 #
##############
# Parameters
n_iter = 1000
lr = 1e-3

# Modified MSE for data with multiple lr values
def mMSE(y, y_bar):
    return np.sum((y - y_bar) ** 2, axis=0)

# Modified Gradient Descent
def mgd(x, y, n_iter, lr):
    # The parameters are now an array that with each value gets closer to the ground truth
    N = 5 # Steps to get to ground truth
    w_0 = np.linspace((true_w0 - 50) * 1e2, true_w0, N)
    w_1 = np.linspace((true_w1 - 50) * 1e2, true_w1, N)
    init = [] # Save initial values
    init.append(np.linspace((true_w0 - 50) * 1e2, true_w0, N))
    init.append(np.linspace((true_w1 - 50) * 1e2, true_w1, N))
    # Modify input data to repeat every column
    # Each column will be used for a different lr
    x = np.array([x] * N).T
    y = np.array([y] * N).T
    # Store loss for each iteration and initial values
    loss = np.zeros((n_iter, N)) 
    # Perform gradient descent
    for i in range(n_iter):
        # 1. Calculate the predictions
        y_bar = w_1 * x + w_0
        # 2. Compute the loss
        loss[i, :] = mMSE(y, y_bar)
        # 3. Calculate the gradients
        dw_1 = 2 * np.sum((y_bar - y) * x, axis=0)
        dw_0 = 2 * np.sum(y_bar - y, axis=0)
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0
    return init[0], init[1], loss # Return only the loss for plots

w_0, w_1, loss_hist = mgd(x, y, n_iter, lr)
# Remove divergent values such as inf, -inf or NaN (which give the warning)
loss_plot = np.where(~np.isfinite(loss_hist), None, loss_hist) 
# create a new plot with a title and axis labels
p = figure(title="Convergence History", x_axis_label='Iteration', y_axis_label='Error', x_range=[0, int(n_iter*0.1)], y_range=[0,4e7])
colors = itertools.cycle(palette) # list of colors
for n in range(len(w_0)):
    label = "w0 = " + str(round(w_0[n], 3)) + ", w1 = " + str(round(w_1[n], 3))
    p.line(np.arange(n_iter), loss_hist[:,n], line_width=4, line_alpha=0.6, legend_label=label, color=next(colors))
plot = (center_plot(p))
plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'3f12f55a-7ba0-4ce7-9f1e-3e21895c2f3f': {'version…

In [183]:
##############
# EXERCISE 3 #
##############

# Parameters
n_iter = 1000
lr = 0.001

# Gradient Descent
def GD(x, y, n_iter, lr):
    # Initialize the parameters
    w_1 = np.random.randn()
    w_0 = np.random.randn()
    loss = np.zeros(n_iter)
    # Perform gradient descent
    for i in range(n_iter):
        # 1. Calculate the predictions
        y_bar = w_1 * x + w_0
        # 2. Compute the loss
        loss[i] = mse_loss(y, y_bar)
        # 3. Calculate the gradients
        dw_1 = 2 * np.sum((y_bar - y) * x)
        dw_0 = 2 * np.sum((y_bar - y))
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0
    return loss

# Stochastic Gradient Descent
def SGD(x, y, n_iter, lr):
    # Initialize the parameters
    w_1 = np.random.randn()
    w_0 = np.random.randn()
    loss = np.zeros(n_iter)
    # Perform gradient descent
    for i in range(n_iter):
        # 1. Calculate the predictions
        y_bar = w_1 * x + w_0
        # 2. Compute the loss
        loss[i] = mse_loss(y, y_bar)
        # 3. Calculate the gradients for a single random value
        index = np.random.randint(0, len(x))
        dw_1 = 2 * np.sum((y_bar[index] - y[index]) * x[index])
        dw_0 = 2 * np.sum((y_bar[index] - y[index]))
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0
    return loss

# Mini-Batch Gradient Descent
def MBGD(x, y, n_iter, lr):
    # Initialize the parameters
    w_1 = np.random.randn()
    w_0 = np.random.randn()
    loss = np.zeros(n_iter)
    # Mini batch is 10% of the actual data
    mb_size = int(len(x) * 0.1)
    
    # Perform gradient descent
    for i in range(n_iter):
        # randomly choose data for batch
        mb_i = np.arange(len(x)) # index list
        np.random.shuffle( mb_i ) # shuffle
        mb_i = mb_i[:mb_size] # pick batch
        mb_x = x[mb_i]
        mb_y = y[mb_i]

        # 1. Calculate the predictions
        y_bar = w_1 * x + w_0 # For real loss
        mb_ybar = w_1 * mb_x + w_0 # For batch
        # 2. Compute the loss
        loss[i] = mse_loss(y, y_bar)
        # 3. Calculate the gradients for a smaller batch
        dw_1 = 2 * np.sum((mb_ybar - mb_y) * mb_x)
        dw_0 = 2 * np.sum((mb_ybar - mb_y))
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0
    return loss

# Evaluate performance by running each function a 100 times
import timeit

SETUP = '''
import numpy as np
true_w0 = 2
true_w1 = 3
x_min = 0
x_max = 1
n_obs = 100
x = np.linspace(x_min, x_max, n_obs)
y = true_w1 * x + true_w0

# Parameters
n_iter = 1000
lr = 0.001

# Gradient Descent
def GD(x, y, n_iter, lr):
    # Initialize the parameters
    w_1 = np.random.randn()
    w_0 = np.random.randn()
    # Perform gradient descent
    for i in range(n_iter):
        # 1. Calculate the predictions
        y_bar = w_1 * x + w_0
        # 3. Calculate the gradients
        dw_1 = 2 * np.sum((y_bar - y) * x)
        dw_0 = 2 * np.sum((y_bar - y))
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0

# Stochastic Gradient Descent
def SGD(x, y, n_iter, lr):
    # Initialize the parameters
    w_1 = np.random.randn()
    w_0 = np.random.randn()
    # Perform gradient descent
    for i in range(n_iter):
        index = np.random.randint(0, len(x))
        # 1. Calculate the predictions
        y_bar = w_1 * x[index] + w_0
        # 3. Calculate the gradients for a single random value
        dw_1 = 2 * np.sum((y_bar - y[index]) * x[index])
        dw_0 = 2 * np.sum((y_bar - y[index]))
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0

# Mini-Batch Gradient Descent
def MBGD(x, y, n_iter, lr):
    # Initialize the parameters
    w_1 = np.random.randn()
    w_0 = np.random.randn()
    # Mini batch is 10% of the actual data
    mb_size = int(len(x) * 0.1)
    
    # Perform gradient descent
    for i in range(n_iter):
        # randomly choose data for batch
        mb_i = np.arange(len(x)) # index list
        np.random.shuffle( mb_i ) # shuffle
        mb_i = mb_i[:mb_size] # pick batch
        mb_x = x[mb_i]
        mb_y = y[mb_i]

        # 1. Calculate the predictions
        mb_ybar = w_1 * mb_x + w_0 # For batch
        # 3. Calculate the gradients for a smaller batch
        dw_1 = 2 * np.sum((mb_ybar - mb_y) * mb_x)
        dw_0 = 2 * np.sum((mb_ybar - mb_y))
        # 4. Update the parameters
        w_1 -= lr * dw_1
        w_0 -= lr * dw_0
'''

STMT1 = "GD(x, y, n_iter, lr)"
STMT2 = "SGD(x, y, n_iter, lr)"
STMT3 = "MBGD(x, y, n_iter, lr)"

gdtime = timeit.timeit(stmt=STMT1, setup=SETUP, number=100)
sgdtime = timeit.timeit(stmt=STMT2, setup=SETUP, number=100)
mbgdtime = timeit.timeit(stmt=STMT3, setup=SETUP, number=100)
print("Batch Gradient Descent: " + str(gdtime) + " seconds")
print("Stochastic Gradient Descent: " + str(sgdtime) + " seconds")
print("Mini-Batch Gradient Descent: " + str(mbgdtime) + " seconds")

# Evaluate convergence history
MBGDloss = MBGD(x, y, n_iter, lr)
SGDloss = SGD(x, y, n_iter, lr)
GDloss = GD(x, y, n_iter, lr)
# Plot convergence history
iter_axis = np.arange(n_iter)
p = figure(title="Convergence History", x_axis_label='Iteration', y_axis_label='Error', x_range=[0, n_iter])
colors = itertools.cycle(palette) # list of colors
p.line(iter_axis, GDloss, line_width=4, line_alpha=0.6, legend_label="Gradient Descent", color=next(colors))
p.line(iter_axis, SGDloss, line_width=4, line_alpha=0.6, legend_label="Stochastic Gradient Descent", color=next(colors))
p.line(iter_axis, MBGDloss, line_width=4, line_alpha=0.6, legend_label="Mini-Batch Gradient Descent", color=next(colors))
plot = (center_plot(p))
plot

Batch Gradient Descent: 1.46258859988302 seconds
Stochastic Gradient Descent: 1.74031630018726 seconds
Mini-Batch Gradient Descent: 2.676761500071734 seconds


BokehModel(combine_events=True, render_bundle={'docs_json': {'b596eae6-1748-4371-b865-9d2e3512f03e': {'version…

## Discovering automatic differentiation - `JAX`

The previous implementation required us to perform manual differentiation of our loss function to understand how to update the parameters. However, large developments have been made in the field of **automatic differentiation**. 

The recent library [JAX](https://github.com/google/jax) extends NumPy with this automatic differentiation feature (*autograd*), while providing a functional approach to numerical computing, allowing for easy gradient computation and just-in-time (JIT) compilation. Its ability to handle complex and custom gradients makes JAX particularly well-suited for advanced research projects. Similar to NumPy, we strongly encourage you to learn JAX through the set of [tutorials](https://jax.readthedocs.io/en/latest/), but we will also provide here the explanation for all of the functions that we will use in this first exercise. After that, we will assume that knowledge of JAX should be found online. 

Note that `JAX` has been thought as an extension of `NumPy`, therefore an extremely large portion of its API simply mirrors the `NumPy` functions by adding automatic differentiation features to it.

**Used functions**
- `random.PRNGKey`: function to generate a key for the pseudorandom number generator. [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.random.PRNGKey.html)

In [184]:
import jax.numpy as jnp
from jax import grad, jit, random
key = random.PRNGKey(42)

### Dataset.

We can simply keep our previous approach to generating the dataset but we will rely on `jnp.ndarray` instead of `np.ndarray`. We also provide the code for splitting the dataset between a `training` and `validation` dataset, as discussed in the course

**Used functions**
- `random.split`: function to split a PRNGKey into a list of subkeys [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.random.split.html)
- `random.uniform`: function to generate an array of uniformly distributed random numbers [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.random.uniform.html)
- `random.normal`: function to generate an array of normally distributed random numbers [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.random.normal.html)

In [185]:
true_w0 = 2
true_w1 = 3
key_0, key_1, key = random.split(key, 3)
x = random.uniform(key_0, (100,))
y = true_w0 + true_w1 * x + (random.normal(key_1, (100,)) * 0.2)

In [186]:
# Splitting our dataset
train_size = int(0.8 * len(x))
x_train, x_valid = x[:train_size], x[train_size:]
y_train, y_valid = y[:train_size], y[train_size:]

### Loss function

Similarly Define the cost function and its gradient.

**Used functions**
- `jnp.mean`: function to compute the mean of an array [Documentation](https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.mean.html)

In [187]:
def model_loss(x, y, w_1, w_0):
    y_bar = x * w_1 + w_0
    return jnp.sum((y - y_bar) ** 2)

In [188]:
w_1 = np.random.randn()
w_0 = np.random.randn()
grad(model_loss)(x, y, w_1, w_0)

Array([-1.2484212 , -1.0721848 , -1.0898596 , -0.9070617 , -0.8293812 ,
       -1.1683482 , -0.86877525, -0.75363934, -0.9773225 , -0.7905905 ,
       -1.4584777 , -1.5343629 , -0.76670915, -1.520057  , -0.9852216 ,
       -1.3344799 , -1.0922543 , -1.1698939 , -1.1750143 , -0.811205  ,
       -1.1504745 , -1.4295597 , -0.9081698 , -1.3798243 , -1.0571618 ,
       -0.9868505 , -1.4148754 , -1.4103822 , -0.9813704 , -1.115773  ,
       -1.3611802 , -0.9324443 , -1.4123068 , -1.035918  , -1.2249516 ,
       -1.0261766 , -1.2809855 , -1.1786911 , -1.1853209 , -0.9810903 ,
       -1.3564099 , -1.4054431 , -1.2243224 , -1.2227871 , -1.0449039 ,
       -1.5152985 , -1.2553107 , -1.2877074 , -1.4470034 , -0.9624081 ,
       -1.3284986 , -1.4827871 , -1.3115807 , -0.94727904, -0.8763239 ,
       -1.1060524 , -1.4351171 , -1.5128423 , -1.459845  , -1.5330809 ,
       -1.2018672 , -1.5669638 , -1.43852   , -1.09085   , -0.857123  ,
       -0.74566495, -1.4746494 , -1.4365221 , -1.2120137 , -1.28

In [189]:
def loss_function(x, y, w_1, w_0):
    y_pred = x * w_1 + w_0
    residuals = y_pred - y
    return jnp.mean(residuals**2)


**Used functions**
- `jit`: function to compile a function for faster execution [Documentation](https://jax.readthedocs.io/en/latest/jit.html)
- `grad`: function to compute the gradient of a function [Documentation](https://jax.readthedocs.io/en/latest/jax.html#jax.grad)

In [190]:
grad_loss_function = jit(grad(loss_function, argnums=[2, 3]))

### Implement the gradient descent algorithm.

Note that nowhere in the code do we need to explicitly define the gradients of the different variables. Yet, the optimization will be performed adequately.

In [191]:
def gradient_descent(key, x, y, lr=0.05, n_iter=1000):
    m = x.shape
    k_0, k_1 = random.split(key, 2)
    w_0 = random.normal(k_0, (1,))
    w_1 = random.normal(k_1, (1,))
    loss_history = []
    for _ in range(n_iter):
        gradients = grad_loss_function(x, y, w_1, w_0)
        #print(gradients[0])
        w_1 -= lr * gradients[0]
        w_0 -= lr * gradients[1]
        loss_history.append(loss_function(x, y, w_1, w_0))
    return w_1, w_0, loss_history

key_gd, key = random.split(key, 2)
w_1, w_0, loss_history = gradient_descent(key_gd, x_train, y_train)
print("Gradient Descent coefficients:", w_1, w_0)

Gradient Descent coefficients: [3.0639796] [1.9661585]


### Evaluate our performances

Thanks to our previous split of the dataset, we are now able to evaluate the performances of our model on *unseen data*, which is the overarching goal of building such machine learning models.

In [192]:
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

y_valid_gd = w_1 * x_valid + w_0
mse_gd = mean_squared_error(y_valid, y_valid_gd)
print("MSE for Gradient Descent:", mse_gd)

MSE for Gradient Descent: 0.03851322


### Visualize the results.

As previously we can witness our results, and also evaluate our solution with our interactive solver.

In [193]:
p = scatter(np.array(x), np.array(y), title="Linear regression")
p.line(np.array(x), np.array(w_1 * x + w_0), line_width=6, line_alpha=0.6, color="red", legend_label=r"Learned model")
plot = (center_plot(p))
plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'258a1df4-2172-4d33-88af-ffeae7f8a8b0': {'version…

In [194]:
from cml.tasks import RegressionLinearSolver
explorer = RegressionLinearSolver()
global key
key = random.PRNGKey(23)
def solve(x, y):
    global key
    key_gd, key = random.split(key, 2)
    w_1, w_0, loss_history = gradient_descent(key_gd, x, y)
    x_predict = np.linspace(np.min(x), np.max(x), 200)
    y_model = w_1 * x_predict + w_0
    return np.array(x_predict), np.array(y_model) #np.array(jnp.zeros(y_model.shape))

explorer.solve = solve
explorer.render()

BokehModel(combine_events=True, render_bundle={'docs_json': {'f6660172-ff51-4bbc-a6af-1e962dda6c9f': {'version…

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Going further

> You are encouraged to experiment in a similar way as outlined for NumPy with different learning rates, maximum number of iterations, and regularization techniques, as well as compare the performance (both accuracy and speed) of the implemented algorithms with `scikit-learn` built-in linear regression functions.

</div>

In [195]:
######################
# YOUR CODE GOES HERE
######################

<a id="capacity"></a>
## Understanding model capacity and selection


In real-life problem, we are aiming to find the parameters of a model, but we do not really know what is the _real_ function underlying this process. So what we can decide to select _any_ function of _any_ **capacity** (complexity of the function). One of the problem with that, is that if we have a too simple function, it will _underfit_ (it is not complex enough for our observations). On the opposite end, if we have a function which is too complex, it might be able to _fit through all training points exactly_ ... even though there is noise in our observations ! This is examplified in the following

<img src="images/01_soa_function_families.png" align="center"/>

We can observe this idea and play with it directly by trying to find a function approximating our previous observations with a polynomial function chosen to have a degree inside \([1,2,8]\).



Depending on the _capacity_ of the model, what we can observe is that

- `capacity too low   -> underfitting   : prediction variance >  noise variance`
- `adequate capacity  -> good fit       : prediction variance == noise variance`
- `capacity too high  -> overfitting    : prediction variance <  noise variance`


A similar example can be given for a classification problem in two dimensions as follows

<img src="images/01_underfit.png" align="center"/>


# Exercises

In the following, we define the exercises that you should fill for this session. These go further than what we have seen together in the course, but they are based on the exact same principles, simply with slightly more complex definitions. We provide an overall guideline for successfully implementing each of the exercise.

## Exercise 1 - Linear classification

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> In this exercise, we will implement linear classification using NumPy and JAX. The goal of this exercise is to gain a better understanding of how linear classification works and how to implement it ourselves using our two first libraries of choice, namely NumPy and JAX. To complete this exercise, you will need to have a basic understanding of NumPy and JAX. You can use the resources provided in the previous exercises to learn more about these libraries.

We help you out by first defining a simple linear classification problem to solve

</div>

In [196]:
import numpy as np
from sklearn.datasets import make_blobs
# Properties of the problem
n_observations = 200
noise = 0.2
c1_center = [-2, -1]
c2_center = [2, 1]
# Create points
x_coords, y_class = make_blobs(n_samples=n_observations, centers=[c1_center, c2_center], n_features=2, cluster_std=0.55)
x_data = x_coords + (noise * np.random.randn(n_observations, 2))
#x_data = x_coords
y_classes = y_class
y_classes[y_classes == 0] = -1

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 1.1 - Implement linear classification with NumPy

> 1. Check the training data with two classes that are linearly separable.
> 2. Initialize weights and bias.
> 3. Implement the forward pass of the linear classifier.
> 4. Implement the hinge loss for classification.
> 5. Derive the gradients (backward pass) of the linear classifier.
> 6. Implement gradient descent to optimize the weights and bias.

</div>

In [197]:
################
# QUESTION 1.1 #
################

# 1. Check the training data with two classes that are linearly separable
feat1 = np.array([i[0] for i in x_data]) # separate features for plot
feat2 = np.array([i[1] for i in x_data])
p = figure(title="Data to classify", x_axis_label='Feature 1', y_axis_label='Feature 2')
p.scatter(feat1[y_classes==1], feat2[y_classes==1], legend_label="Class 1", color="blue")
p.scatter(feat1[y_classes==-1], feat2[y_classes==-1], legend_label="Class 2", color="red")
# 2. Initialize weights and bias
w = np.random.randn(3)
# 3. Implement the forward pass of the linear classifier
y_pred = w[0] + x_data @ w[1:]
# 4. Implement the hinge loss for classification
def hinge_loss(y_real, y_pred):
    return np.mean([max(0, 1. - m * n) for m, n in zip(y_real, y_pred)])
# 5. Derive the gradients (backward pass) of the linear classifier
def grad_des(y_real, y_pred):
    dmax = np.sign(1. -y_real * y_pred)
    dmax[dmax==-1] = 0
    return np.array([np.mean(dmax * -y_real), np.mean(dmax * -y_real * feat1), np.mean(dmax * -y_real * feat2)])
# 6. Implement gradient descent to optimize weights and bias
n_iter = 200
lr = 0.01
error = np.zeros(n_iter)
for i in range(n_iter):
    y_pred = w[0] + x_data @ w[1:] # Forward pass
    error[i] = hinge_loss(y_classes, y_pred) # Get error to see if gradient + lr is correct
    gd = grad_des(y_classes, y_pred) # backward pass
    w -= lr * gd # gradient descent

print("Error at last iteration: " + str(error[-1]))
# Attempt to plot boundary decision, not sure if it's correct
x_clf = -np.linspace(np.min(x_data), np.max(x_data), len(y_classes))
x1 = np.linspace(np.min(feat1), np.max(feat1), len(feat1))
x2 = np.linspace(np.min(feat2), np.max(feat2), len(feat2))
y_clf = w[0] + x1 * w[1] + x2 * w[2]
p.line(x_clf, y_clf, line_width=4, line_alpha=0.6, legend_label="Linear Classifier", color="green")
plot = (center_plot(p))
plot

Error at last iteration: 0.1516199941257406


BokehModel(combine_events=True, render_bundle={'docs_json': {'2a4e0cf1-900d-4bd0-b165-13652897cb79': {'version…

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Going further (optional)
> Try to replace the Hinge loss by the cross-entropy loss in your implementation

Not sure if my gradient is correct, although the error seems random it seems to work

</div>

In [198]:
# 1. Check the training data with two classes that are linearly separable
feat1 = np.array([i[0] for i in x_data]) # separate features for plot
feat2 = np.array([i[1] for i in x_data])
p = figure(title="Data to classify", x_axis_label='Feature 1', y_axis_label='Feature 2')
p.scatter(feat1[y_classes==1], feat2[y_classes==1], legend_label="Class 1", color="blue")
p.scatter(feat1[y_classes==-1], feat2[y_classes==-1], legend_label="Class 2", color="red")
# 2. Initialize weights and bias
w = np.random.randn(3)

# needed for loss
def softmax(y):
    exp_values = np.exp(y)
    return exp_values/np.sum(exp_values)
# loss function
def cel(y_real, y_pred):
    y_pred = softmax(y_pred)
    loss = 0
    # cross entropy Loss
    for i in range(len(y_pred)):
        loss = loss - (y_real[i] * np.log(y_pred[i]))

    return loss

def grad_des(y_real, y_pred):
    grad0 = np.sum(softmax(y_real) - softmax(y_pred))
    grad1 = np.sum(softmax(y_real) - softmax(y_pred)*feat1)
    grad2 = np.sum(softmax(y_real) - softmax(y_pred)*feat2)
    return np.array([grad0, grad1, grad2])
    

# 6. Implement gradient descent to optimize weights and bias
n_iter = 200
lr = 0.01
error = np.zeros(n_iter)
for i in range(n_iter):
    y_pred = w[0] + x_data @ w[1:] # Forward pass
    error[i] = cel(y_classes, y_pred) # Get error to see if gradient + lr is correct
    gd = grad_des(y_classes, y_pred) # backward pass
    w -= lr * gd # gradient descent

print("Error at last iteration: " + str(error[-1]))
# Attempt to plot boundary decision, not sure if it's correct
x_clf = -np.linspace(np.min(x_data), np.max(x_data), len(y_classes))
x1 = np.linspace(np.min(feat1), np.max(feat1), len(feat1))
x2 = np.linspace(np.min(feat2), np.max(feat2), len(feat2))
y_clf = w[0] + x1 * w[1] + x2 * w[2]
p.line(x_clf, y_clf, line_width=4, line_alpha=0.6, legend_label="Linear Classifier", color="green")
plot = (center_plot(p))
plot

Error at last iteration: -1656.6037829240133


BokehModel(combine_events=True, render_bundle={'docs_json': {'c7d1039c-d429-4710-a64a-d9d2fed16ef0': {'version…

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 1.2 - Implement linear classification with JAX

> 1. Initialize weights and bias.
> 2. Define the loss function using the jax.nn.softmax_cross_entropy_with_logits function.
> 3. Implement gradient descent to optimize the weights and bias.

</div>

In [199]:
######################
# YOUR CODE GOES HERE
######################

[0.00740265 0.7339437  2.8501277 ]


<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 1.3 - Compare the results of the NumPy and JAX implementations.
> 1. Plot the decision boundary for each implementation.
> 2. Compare the training time and accuracy of each implementation.

</div>

In [200]:
######################
# YOUR CODE GOES HERE
######################

<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

> ### Going further (optional)
> Implement a more complex dataset and compare the results of the NumPy and JAX implementations.

</div>

In [201]:
######################
# YOUR CODE GOES HERE
######################

## Exercise 2 - Polynomial regression

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> In this exercise, you will implement polynomial regression using NumPy and JAX. This should be a quite straightforward extension of what we have seen until now. We deliberately removed details of the implementation (list of points to adress) so that you can start defining your first entire machine learning setup by yourself.

</div>

In [240]:
degree = 3
coefs = np.random.randn(degree)
print("True coefficients: " + str(coefs))
pol = np.poly1d(coefs)
print("Model: " + str(pol))
n_obs = 200
noise = 20
x = np.linspace(-50, 50, n_obs)
y = pol(x) # ground truth
y_obs = y + (noise * np.random.randn(n_obs,))
p = scatter(np.array(x), np.array(y_obs), title="Polynomial regression")
p.line(x, y, line_width=6, line_alpha=0.6, color="green", legend_label=r"Truth")
plot = (center_plot(p))
plot

True coefficients: [1.80034147 0.37962331 0.45661008]
Model:      2
1.8 x + 0.3796 x + 0.4566


BokehModel(combine_events=True, render_bundle={'docs_json': {'0ee9f6e3-5b4e-48a4-82c1-bd74dd63117b': {'version…

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 2.1 - Implement polynomial regression using NumPy with gradient descent.

</div>

In [241]:
n_iter = 200 * degree
lr = 0.001**degree

# define &init model
weights = np.random.randn(degree)
model = np.poly1d(weights)

def loss(y_real, y_pred):
    return np.mean((y_pred - y_real)**2)

def grad(y_real, y_pred):
    return 2. / len(y_real) * np.sum(y_pred - y_real)

error = np.zeros(n_iter)
for i in range(n_iter):
    # forward
    model = np.poly1d(weights[::-1])
    y_pred = model(x)
    # error
    error[i] = loss(y_obs, y_pred)
    # backward
    weights[0] -= lr * grad(y_obs, y_pred)
    for j in range(1, degree):
        weights[j] -= lr * grad(y_obs, y_pred) * np.sum(x**j)

print("Error at last iteration: " + str(error[i]))
print("True coefficients: " + str(coefs))
print("Trained weights: " + str(weights[::-1]))
p = scatter(np.array(x), np.array(y_obs), title="Polynomial regression")
p.line(x, y, line_width=6, line_alpha=0.6, color="green", legend_label=r"Truth")
p.line(x, y_pred, line_width=6, line_alpha=0.6, color="red", legend_label=r"Model")
plot = (center_plot(p))
plot

Error at last iteration: 452.5062377407853
True coefficients: [1.80034147 0.37962331 0.45661008]
Trained weights: [1.80032802 0.20945004 0.46731708]


BokehModel(combine_events=True, render_bundle={'docs_json': {'327805c1-9480-4607-83e4-36b903a27ad2': {'version…

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 2.2. Implement polynomial regression using JAX with gradient descent.

</div>

In [243]:
######################
# YOUR CODE GOES HERE
######################

<div class="alert alert-success" markdown="1" style="color:white; background-color: #013220; border-color: #03C03C">

> ### Question 2.3 - Compare the results of both implementations and the speed of the optimization algorithm.

</div>

In [205]:
######################
# YOUR CODE GOES HERE
######################

<a id="audio"></a>
# Audio applications

In order to test our algorithms on audio and music data, we will work with several datasets. We will both be using well-known state-of-art datasets (through the TF dataset system). But first, we will rely on a simple dataset (`Musclefish`) that should be downloaded on your local computer first from this [link](https://nubo.ircam.fr/index.php/s/ByK4QL7nE4Mq5MA)

**The following set of instructions will perform that automatically for you :)**

In [206]:
!curl https://nubo.ircam.fr/index.php/s/ByK4QL7nE4Mq5MA/download/musclefish.zip --output musclefish.zip
!unzip musclefish.zip
!mkdir data
!mv musclefish data/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 13 3008k   13  408k    0     0   244k      0  0:00:12  0:00:01  0:00:11  244k
 56 3008k   56 1696k    0     0   633k      0  0:00:04  0:00:02  0:00:02  634k
 84 3008k   84 2536k    0     0   692k      0  0:00:04  0:00:03  0:00:01  692k
100 3008k  100 3008k    0     0   750k      0  0:00:04  0:00:04 --:--:--  750k
'unzip' is not recognized as an internal or external command,
operable program or batch file.
A subdirectory or file data already exists.
'mv' is not recognized as an internal or external command,
operable program or batch file.


## Dataset details

  |**Type**|*Origin*|
  |-------:|:---------|
  |**Classification**|[*MuscleFish*](http://knight.cis.temple.edu/~vasilis/Courses/CIS750/Papers/muscle_fish.pdf) dataset|
  |**Music-speech**|[*MIREX Recognition*](http://www.music-ir.org/mirex/wiki/2015:Music/Speech_Classification_and_Detection) set|
  |**Source separation**|[*SMC Mirum*](http://smc.inesctec.pt/research/data-2/) dataset|
  |**Speech recognition**|[*CMU Arctic*](http://festvox.org/cmu_arctic/) dataset|

**Unzip the file and place the `musclefish` folder inside a `data` folder in the root of this notebook**
For the first parts of the tutorial, we will mostly rely solely on the classification dataset. In order to facilitate the interactions, we provide a dataset class called `AudioSupervisedDataset` that will allow to import all audio datasets along the tutorials. This class also contains a set of holders and will allow us to perform batch-wise gradient descent.

```Python
class AudioSupervisedDataset():
    """
    Helper class to import datasets
    % class_path  : Path to the dataset (string)
    % type       : Type of dataset (string: 'classify', 'plain', 'metadata')
    """ 
    audio_files
    labels
    labels_names
    num_examples 
    num_classes 
```

We also provide a simplified function `import_dataset` that is demonstrated below.

***
<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

**_Exercise_**  

  1. Launch the import procedure and check the corresponding structure
  2. Code a count function that prints the name and number of examples for each classes 

</div>

***

In [207]:
from cml.data import import_dataset, AudioSupervisedDataset
data_path = "data/"
batch_size = 16
# Instantiate the dataset class
train_dataset = import_dataset(data_path, "musclefish", 'train', batch_size=batch_size)
# Generate a batch of data from the train dataset
batch_x, batch_y = next(train_dataset)
# Print the shape of the input and output batch tensors
print(batch_x.shape)  # (16, 44100)
print(batch_y.shape)  # (16,)

Recursive search in data//musclefish


IndexError: list index out of range

In [ ]:
#%% Q-0.1.2 - Count function to print the number of examples in each class along with class label

n_batch = 0
train_dataset._reset_generator()
for batch_x, batch_y in iter(train_dataset):
    n_batch += 1
    ######################
    # YOUR CODE GOES HERE
    ######################

print(n_batch)

52


### Preprocessing

We will rely on a set of spectral transforms that allow to obtain a more descriptive view over the audio information. As most of these are out of the scope of the machine learning course, we redirect you to a [signal processing course](https://ccrma.stanford.edu/~jos/sasp/) proposed by [Julius O. Smith](https://ccrma.stanford.edu/~jos/).  

The following functions to compute various types of transforms are given as part of the basic audio dataset class, in the `cml.data.audio` package

  |**Name**|*Transform*|
  |-------:|:----------|
  |`stft`       |[Short-term Fourier transform](https://en.wikipedia.org/wiki/Short-time_Fourier_transform)|
  |`mel`  |[Mel scale](https://en.wikipedia.org/wiki/Mel_scale) transform|
  |`chroma` |[Chromas vector](https://en.wikipedia.org/wiki/Harmonic_pitch_class_profiles)|
  |`cqt`        |[Constant-Q](https://en.wikipedia.org/wiki/Constant_Q_transform) transform|

In order to perform the various computations, we provide the following function, in the `AudioSupervisedDataset` which performs the different transforms on a complete dataset.  

``` Python
dataset.transform(index, name)
    """ index   : Specific index in our dataset """
    """ name    : Name of the transform to apply """

# The name can be selected in 
"stft"      # Power spectrum (STFT)
"mel"       # Spectrum in Mel scale
"chroma"    # Chroma vectors
"cqt"       # Constant-Q transform
```


***
<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

**Exercise**  

  1. Launch the transform computation procedure and check the corresponding structure
  2. For each class, select a random element and plot its various transforms on a single plot. You should obtain plots similar to those shown afterwards.
  3. For each transform, try to spot major pros and cons of their representation.
  
</div>

***

In [ ]:
# 0.2 - Pre-process the audio to obtain spectral transforms 
power_spec = train_dataset.transform(0, "stft")

In [ ]:
#%% Q-0.2.2 - Plot the various transforms 

######################
# YOUR CODE GOES HERE
######################


### Features

<div markdown = "1">

As you might have noted from the previous exercice, most spectral transforms have a very high dimensionality, and might not be suited to exhibit the relevant structure of different classes. To that end, we provide a set of functions for computing several spectral features in the `cml.data` package, we redirect interested readers to this [exhaustive article](http://recherche.ircam.fr/anasyn/peeters/ARTICLES/Peeters_2003_cuidadoaudiofeatures.pdf) on spectral features computation.

  |**File**|*Transform*|
  |-------:|:----------|
  |`spectral_centroid`|Spectral centroid|
  |`spectral_bandwidth`|Spectral bandwidth|
  |`spectral_contrast`|Spectral contrast|
  |`spectral_flatness`|Spectral flatness|
  |`spectral_rolloff`|Spectral rolloff|

Once again, we provide a function to perform the computation of different features on a complete set. Note that for each feature, we obtain the temporal evolution in a vector. Therefore, for further learning tasks, if you wish to obtain simplified spaces, you might need to compute the mean and standard deviation of each feature.

``` Python
dataset.feature(index, name)
    """ index   : Specific index in our dataset """
    """ name    : Name of the feature to obtain """

# Name can be chosen inside the following
"loudness"    # Loudness
"centroid"    # Spectral centroid
"bandwidth"   # Spectral bandwidth
"contrast"    # Spectral contrast
"flatness"    # Spectral flatness
"rolloff"     # Spectral rolloff

```

***
<div class="alert alert-success" markdown="1" style="color:white; background-color: #192841; border-color: #779ecb">

**Exercise**

  1. Launch the feature computation procedure and check the corresponding structure
  2. This time for each class, superimpose the plots of various features on a single plot, along with a boxplot of mean and standard deviations. You should obtain plots similar to those shown afterwards.
  3. What conclusions can you make on the discriminative power of each feature ?
  4. Perform scatter plots of the mean features for all the dataset, while coloring different classes.
  5. What conclusions can you make on the discriminative power of mean features ?

</div>

***

In [ ]:
# 0.3 - Compute temporal spectral features
power_spec = train_dataset.feature(0, "centroid")

In [ ]:
#%% Q-0.3.2 - Plot the various features 

# Use these styles for boxplot
boxprops=dict(linewidth=3, color='white')
whiskerprops=dict(linewidth=3, color='white')
medianprops=dict(linewidth=2.5, color='firebrick')
flierprops = dict(markeredgecolor='white', markerfacecolor='firebrick')

######################
# YOUR CODE GOES HERE
######################


In [ ]:
#%% Q-0.3.4 - Observe the distribution of classes for different features

# This allows to use 3D rendering in matplotlib
from mpl_toolkits.mplot3d import Axes3D
plt.figure(figsize=(12,8))
# Create a vector of random colors for each class
colorVect = np.zeros((3, len(data_struct["class_names"])));
for c in range(len(data_struct["class_names"])):
    colorVect[:,c] = np.random.rand(3);

######################
# YOUR CODE GOES HERE
######################


That's it for this tutorial, now remember that we can use any form of description (features) as a basis for learning algorithms. We will see in the next tutorial what we an do with these features.